In [2]:
import pandas as pd
import datetime
import requests as req
import numpy as np

In [3]:
path = r"\\home.rrze.uni-erlangen.de\douk11ibag\Documents\Python\big data\athlete_events.csv"

In [4]:
df = pd.read_csv(path)
print(len(df))
df = df[df['Year'] >= 1980]

271116


# Target

NOC merge

Games Title erstellen -> 1900 Summergames // Zusatz 1900 Summer Paris

# Year Season City
merge with two fields

In [5]:
year_season_city = df[['Year', 'Season', 'City']]
year_season_city = year_season_city.drop_duplicates()
year_season_city = year_season_city.sort_values(['Year'], ascending=False)
year_season_city = year_season_city.reset_index(drop=True)
year_season_city.sample(4)

,Year,Season,City
13,1992,Summer,Barcelona
0,2016,Summer,Rio de Janeiro
18,1980,Winter,Lake Placid
2,2012,Summer,London


In [6]:
year_season_city.to_csv(r"\\home.rrze.uni-erlangen.de\douk11ibag\Documents\Python\big data\year_season_city.csv")

# Noc
left noc and right noc_id, some nocs are missing -> left join!

In [21]:
noc = df[['NOC', 'Team']].drop_duplicates().reset_index(drop=True)

# Athlets
left on ID, right on Index

In [8]:
athleths = df[['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight']]
athleths = athleths.drop_duplicates(['ID'])
athleths = athleths.set_index('ID',drop=True)

# übung: athleths['Name'].str.rsplit(' ', n=1)

athleths

,Name,Sex,Age,Height,Weight
ID,,,,,
1,A Dijiang,M,24.0,180.0,80.0
2,A Lamusi,M,23.0,170.0,60.0
5,Christine Jacoba Aaftink,F,21.0,185.0,82.0
6,Per Knut Aaland,M,31.0,188.0,75.0
7,John Aalberg,M,31.0,183.0,72.0
9,Antti Sami Aalto,M,26.0,186.0,96.0
11,Jorma Ilmari Aalto,M,22.0,182.0,76.5
12,Jyri Tapani Aalto,M,31.0,172.0,70.0
13,Minna Maarit Aalto,F,30.0,159.0,55.5


# Etherium Preise

In [9]:
resp = req.get('https://query1.finance.yahoo.com/v8/finance/chart/BTC-EUR?symbol=BTC-EUR&period1=1557342840&period2=1557580689&interval=15m&includePrePost=true&events=div|split|earn&lang=de-DE&region=DE&crumb=NFWlGk6OV6P&corsDomain=de.finance.yahoo.com')

d = resp.json()
dt = d['chart']['result'][0]['timestamp']
df_stock = {'timestamp':dt}

for k in d['chart']['result'][0]['indicators']['quote'][0].keys():
    df_stock[k] = d['chart']['result'][0]['indicators']['quote'][0][k]
    
df_stock = pd.DataFrame(df_stock)
min15 = 60*15
df_stock['timestamp'] = df_stock['timestamp'] + (min15 - np.random.random(len(df_stock)) * min15 * 2).astype(int)
df_stock['timestamp'] = pd.to_datetime(df_stock['timestamp'], unit='s')
df_stock = df_stock.sort_values('timestamp')
df_stock.sample(10)

,timestamp,close,high,open,low,volume
22,2019-05-09 00:41:49,5333.520020,5341.209961,5321.209961,5321.209961,257714
162,2019-05-10 11:15:08,5617.529785,5619.790039,5608.890137,5608.430176,1068060
240,2019-05-11 06:51:59,6056.580078,6065.359863,6063.629883,6036.740234,1447556
161,2019-05-10 11:07:45,5608.640137,5610.770020,5610.189941,5606.089844,901344
101,2019-05-09 20:03:30,5417.529785,5425.000000,5420.209961,5417.529785,308940
127,2019-05-10 02:37:40,5552.970215,5575.490234,5561.200195,5546.870117,718342
11,2019-05-08 21:43:45,5285.669922,5297.060059,5291.229980,5285.669922,208016
186,2019-05-10 17:41:16,5701.450195,5708.180176,5702.520020,5700.040039,755952
73,2019-05-09 13:21:01,5357.169922,5368.620117,5353.200195,5345.500000,840724
149,2019-05-10 08:19:09,5614.649902,5633.100098,5632.290039,5608.740234,1707816


In [10]:
dtu = df_stock['timestamp'].dt.date.unique()

In [11]:
df_open = pd.DataFrame({'Date':dtu})
df_open['key'] = 0
df_open['Location'] = 'FRA'

df_opening_times = pd.DataFrame({'Time': [8, 22]})
df_opening_times['key'] = 0

df_market = pd.merge(df_open, df_opening_times, on='key')
df_market = df_market.drop(columns=['key'])

df_market['Date'] = df_market['Date'] + df_market['Time'].apply(lambda x: pd.DateOffset(hours=x))

In [12]:
# solution
df_solution = pd.merge_asof(df_market, df_stock, left_on='Date', right_on='timestamp', direction='nearest')
df_solution

,Date,Location,Time,timestamp,close,high,open,low,volume
0,2019-05-08 08:00:00,FRA,8,2019-05-08 18:45:25,5272.580078,5274.000000,5273.229980,5272.459961,0
1,2019-05-08 22:00:00,FRA,22,2019-05-08 22:05:34,5280.270020,5292.149902,5289.560059,5280.120117,276932
2,2019-05-09 08:00:00,FRA,8,2019-05-09 08:06:33,5442.979980,5444.540039,5428.810059,5428.810059,558388
3,2019-05-09 22:00:00,FRA,22,2019-05-09 21:55:10,5482.720215,5484.129883,5464.439941,5458.529785,991024
4,2019-05-10 08:00:00,FRA,8,2019-05-10 07:58:59,5632.410156,5656.069824,5642.930176,5620.810059,1909982
5,2019-05-10 22:00:00,FRA,22,2019-05-10 21:56:03,5648.859863,5655.370117,5652.060059,5642.729980,346088
6,2019-05-11 08:00:00,FRA,8,2019-05-11 08:13:35,6017.609863,6062.890137,6052.459961,6016.379883,2288096
7,2019-05-11 22:00:00,FRA,22,2019-05-11 13:00:10,6044.529785,6049.890137,6049.640137,6043.830078,86224


In [13]:
# median columnwise
df_solution['median'] = df_solution[['open', 'close']].median(axis=1)
df_solution

,Date,Location,Time,timestamp,close,high,open,low,volume,median
0,2019-05-08 08:00:00,FRA,8,2019-05-08 18:45:25,5272.580078,5274.000000,5273.229980,5272.459961,0,5272.905029
1,2019-05-08 22:00:00,FRA,22,2019-05-08 22:05:34,5280.270020,5292.149902,5289.560059,5280.120117,276932,5284.915039
2,2019-05-09 08:00:00,FRA,8,2019-05-09 08:06:33,5442.979980,5444.540039,5428.810059,5428.810059,558388,5435.895020
3,2019-05-09 22:00:00,FRA,22,2019-05-09 21:55:10,5482.720215,5484.129883,5464.439941,5458.529785,991024,5473.580078
4,2019-05-10 08:00:00,FRA,8,2019-05-10 07:58:59,5632.410156,5656.069824,5642.930176,5620.810059,1909982,5637.670166
5,2019-05-10 22:00:00,FRA,22,2019-05-10 21:56:03,5648.859863,5655.370117,5652.060059,5642.729980,346088,5650.459961
6,2019-05-11 08:00:00,FRA,8,2019-05-11 08:13:35,6017.609863,6062.890137,6052.459961,6016.379883,2288096,6035.034912
7,2019-05-11 22:00:00,FRA,22,2019-05-11 13:00:10,6044.529785,6049.890137,6049.640137,6043.830078,86224,6047.084961


In [14]:
# prüfsumme übung
df_solution['median'].sum()

44837.545166015625